In [34]:
from pathlib import Path
from tqdm import tqdm#_notebook as tqdm
from pku_autonomous_driving import io, util, dataset, resnet, centernet, prediction, graphics

import importlib
importlib.reload(io)
importlib.reload(util)
importlib.reload(dataset)
importlib.reload(resnet)
importlib.reload(centernet)
importlib.reload(prediction)
importlib.reload(graphics)

<module 'pku_autonomous_driving.graphics' from '/Users/argon/workspace/managed/pku-autonomous-driving/pku_autonomous_driving/graphics.py'>

In [29]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
test = io.load_test_data()
test_loader = prediction.create_data_loader(test)

In [31]:
base_model = resnet.resnext50_32x4d(pretrained=False)
model = centernet.CentResnet(base_model, 8)
util.setup_model(model, device, Path("../input/pkuautonomousdrivingtraining/resnext50.pth"))

In [32]:
%%time
predictions = []
model.eval()

for idx, (img, _, _) in tqdm(enumerate(test_loader)):
    org_shape = io.load_image(test_loader.dataset.dataset[idx].image_id, training=False).shape
    prediction.clean_up()
    result = prediction.predict(model, img.to(device), org_shape)
    predictions = [*predictions, *result]


0it [00:00, ?it/s]
1it [00:21, 21.46s/it]
2it [00:43, 21.69s/it]

KeyboardInterrupt: 

In [37]:
test = io.load_test_data_raw()
test['PredictionString'] = predictions
test.to_csv('predictions.csv', index=False)
test.head()

ValueError: Length of values does not match length of index